### Load Models and tokenizers with Auto Classes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Maykeye/TinyLlama-v0"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

Training parameters

In [ ]:
from transformers import TrainingArguments 

training_args = TrainingArguments(
    learning_rate=2e-3, 
    warmup_ratio=0.03,
  	num_train_epochs=3,
  	output_dir='/tmp',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    save_steps=10,
    logging_steps=2,
    lr_scheduler_type='constant',
    report_to='none'
)
  

In [ ]:
from trl import SFTTrainer

# Instantiate fine-tuning class
trainer = SFTTrainer(
  model=model,
  tokenizer=tokenizer,
  train_dataset=dataset,
  dataset_text_filed='conversation',
  max_seq_length=250,
  args=training_args
)

In [ ]:
# Start training 
trainer.train()

Evaluate a trainted model

In [ ]:
import evaluate

rogue = evaluate.load('rouge')
predictions = ["hello there", "neneral kenobi"]
references = ["hello there", "master yoda"]
results = rogue.compute(predictions=predictions, references=references)
print(results)

In [ ]:
def generate_predictions_and_reference(dataset):
  predictions=[]
  references=[]
  for row in dataset:
    inputs = tokenizer.encode(row["instruction"], return_tensors="pt")
    outputs = model.generate(inputs)
    decoded_output = tokenizer.decode(outputs[0, inputs.shape[1]:], skip_special_tokens=True)
    references += [row["response"]]
    predictions += [decoded_output]
    
  return predictions, references

In [ ]:
references, predictions = generate_predictions_and_reference(evauluation_dataset)


In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predictions, references=references)